---

**Overview of This Notebook**

This notebook provides an introduction to LangChain, along with a few examples demonstrating how to use LangChain with AWS Bedrock models.

---

### **What is LangChain?**

LangChain is a powerful framework designed to simplify the development of applications powered by large language models (LLMs).

---

### **Benefits of Using LangChain**

1. Offers a comprehensive set of tools to build LLM-based applications.
2. Provides common, reusable functionalities such as prompt generation.
3. Enables seamless integration with external data sources, databases, and third-party tools.
4. Supports experimentation with different LLMs, allowing users to evaluate and choose models based on specific performance requirements.

---

### **Alternative to LangChain**

* **Together AI** – A platform that hosts a wide range of language models in one centralized location, providing an alternative approach to building LLM-based solutions.

---

### **Additional Tools Supported in LangChain**

1. **LangSmith** – A tool for tracing and evaluating LLM applications and intelligent agents, helping developers transition from prototyping to production.
2. **LangGraph** – Used to build stateful, multi-agent applications with LLMs. While it integrates seamlessly with LangChain, it can also function independently. LangGraph is trusted by companies like LinkedIn, Uber, Klarna, GitLab, and others for production-level deployment.



**Install Required Dependencies**

To run this notebook, ensure that all necessary dependencies are installed.

If you plan to use AWS Bedrock models, make sure your AWS credentials are configured beforehand. You can set them up using environment variables as described in the AWS CLI documentation:
🔗 [Configure AWS Credentials](https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-envvars.html)



While running in vscode, open the terminal in vscode, navigate to ~/.aws/credentials, copy the credentials under the default and restart the kernel.

In [2]:
!python -m pip install awscli langchain langchain-aws

  Using cached awscli-1.40.43-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [awscli]2m4/5 [awscli]s]


With the environment set up and all dependencies installed, we're ready to start loading models and exploring LangChain.

---

### **Overview of LangChain Chat Models**

LangChain's chat models offer a convenient interface to interact with modern large language models (LLMs). These models come with enhanced capabilities that go beyond simple text generation.

#### **Key Capabilities of Chat Models**

1. Tool invocation (e.g., calling APIs or functions)
2. Structured output generation
3. Support for multimodal inputs (e.g., text, images)

---

### **Core Features of LangChain**

1. Seamless integration with various chat models
2. Built-in message formatting using LangChain’s standardized prompt structure
3. Tool calling API for dynamic function execution
4. Support for structured and typed outputs
5. Efficient batching, asynchronous programming, and a robust streaming API
6. Integration with **LangSmith** for monitoring and debugging
7. Built-in rate limiting and caching mechanisms to optimize performance

---



---

### **Various Methods to Interact with Chat Models**

There are multiple ways to retrieve responses from chat models. Each method should be implemented to explore its functionality:

1. `invoke` – Standard method to get a single response
2. `stream` – Stream responses in real time
3. `batch` – Process multiple inputs in parallel
4. `bind_tools` – Use tools/functions in combination with the model
5. Structured Output – Generate and parse structured responses (e.g., JSON)

For more details and additional methods, refer to the LangChain API documentation:
🔗 [LangChain BaseChatModel Reference](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html)

---



In [3]:
# Ensure your AWS credentials are configured

from langchain.chat_models import init_chat_model

model = init_chat_model("anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '08749c96-e8a5-4593-ba9e-675f98ea649f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 26 Jun 2025 03:14:04 GMT', 'content-type': 'application/json', 'content-length': '185', 'connection': 'keep-alive', 'x-amzn-requestid': '08749c96-e8a5-4593-ba9e-675f98ea649f'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [424]}, 'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run--f5012e57-268e-4912-a6d6-f91d8c58eb2a-0', usage_metadata={'input_tokens': 17, 'output_tokens': 7, 'total_tokens': 24, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})



---

### **Prompt Templates**

LangChain provides support for creating prompt templates that are structured and ready to be passed directly into a language model.

For more details, refer to the documentation:
🔗 [ChatPromptTemplate Reference](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

---




In [5]:
from langchain_core.prompts import ChatPromptTemplate
import pprint
import json

def read_data(filename):
    with open(filename, "r") as f:
        return json.load(f)

prompt_templates = read_data("prompt_templates.json")
query_data = read_data("query_data.json")




In [6]:
def get_prompt(key, parameters):
    # Load the prompt template from the JSON file
    global prompt_templates
    lc_prompt_template = ChatPromptTemplate.from_messages(prompt_templates[key])
    prompt = lc_prompt_template.invoke(parameters)
    return prompt

def invoke_model(model, key="summarization_prompts", parameters=None):
    """
    Invoke the model with a specific prompt key and parameters.
    """
    prompt = get_prompt(key, parameters)
    response = model.invoke(prompt)
    return response

# def invoke_summarization(model):
#     global query_data
#     prompt = get_prompt("summarization_prompts", {"text": query_data["summarization_text"]})
#     response = model.invoke(prompt)
#     return response

# def invoke_information_extraction(model):
#     global query_data
#     prompt = get_prompt("information_extraction_prompts", {"text": query_data["information_extraction_text"]})
#     response = model.invoke(prompt)
#     return response

In [7]:
pprint.pprint(invoke_model(model, "question_answering_prompts", {"text": query_data["question_answering_text"], "question": query_data["question_answering_question"]}).content)

'OKT3 was originally sourced from mice.'




---

### **Summary**

1. Learned how to import models using LangChain and directly invoke them with queries to retrieve responses.
2. Gained an understanding of LangChain’s message format, including `HumanMessage` and `SystemMessage`.
3. Explored various methods for providing context to the model through prompting.
4. Learned how to use LangChain's prompt templating to craft more effective and structured prompts for improved model performance.

---






---

### **More with LangChain**

Next, we’ll explore another approach to prompting that helps generate structured output from the model. To achieve this, we’ll use tools such as `BaseModel` and `Field` from the **Pydantic** library.

Let’s dive deeper into how this works.

---


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

# Ensure your AWS credentials are configured
# using init_chat_model, initializing the aws bedrock anthropic model
model = init_chat_model("anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

# Inorder to use structured output, we need to define a Pydantic model that matches the expected output structure.
# Steps to create a structured output model:
# Define a class that inherits from BaseModel and then define the fields with appropriate types and constraints.
# Use the `with_structured_output` method to wrap the model with the structured output capabilities.
class Classification(BaseModel):
  semantic_label: str = Field(..., description="The semantic label of the classification", enum=["positive", "negative", "neutral"])
  agression_score: float = Field(..., description="The aggression score of the classification, between 0 and 5", ge=0, le=5)
  language: str = Field(..., description="The language of the classification", enum=["en", "it", "fr", "de", "es"])

structured_modal = model.with_structured_output(Classification)

prompt_template = ChatPromptTemplate.from_template(
  "Extract the desired information from the following text: {text}. " \
  "Only extract the properties of the Classification class."
)
lc_prmopt = prompt_template.invoke({"text": "I am so angry that i don't want to meet you at all from the things that you did to me."})

structured_response = structured_modal.invoke(lc_prmopt)

pprint.pprint(structured_response)
print(f"Semantic Label: {structured_response.semantic_label}")
print(f"Aggression Score: {structured_response.agression_score}")
print(f"Language: {structured_response.language}")

Classification(semantic_label='negative', agression_score=4.0, language='en')
Semantic Label: negative
Aggression Score: 4.0
Language: en
